## keras pre-trained resnet50/vgg19

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
keras.__version__

'2.2.4-tf'

In [ ]:
base_model = keras.applications.resnet50.ResNet50(weights='imagenet')

In [ ]:
base_model.summary()

In [2]:
%matplotlib inline
%load_ext tensorboard.notebook

In [ ]:
keras.utils.plot_model(base_model)

In [3]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## archi

In [ ]:
class scalable_img_classifier(object):
    from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    from tensorflow.keras.applications import resnet50, vgg16, mobilenet_v2
    def __init__(self, feature_extrator=None):
        self.feature_extractions = resnet50.ResNet50(weights='imagenet', include_top=False)
        
    def learn(X, y, class_name):
        pass
        
            

In [ ]:
class incre_learning_schemer(object):
    def __init__(self):
        pass
    
    def create_learning_sessions(self, X, y, batch_number=1):
        pass

    def evaluate_past_sessions(self):
        pass

    def train_evaluate(X, y):
        pass
    

In [ ]:
x_train.shape, y_train.shape

### let's train a bin classifier, find what happens when pos/neg samples are extremely unbalanced

In [9]:
x_train_bin = x_train.copy()
y_train_bin = y_train==1
x_train_bin.shape, y_train_bin.shape

((50000, 32, 32, 3), (50000, 1))

In [5]:
from tensorflow.keras.applications.resnet50 import *

In [6]:
x = preprocess_input(x_train_bin[0])

In [7]:
x = np.expand_dims(x, axis=0)
x.shape

(1, 32, 32, 3)

In [4]:
m = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)
m.summary()

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, None, None, 6 256         conv1[0][0]                      
___________________________________________________________________________________________

In [5]:
for layer in m.layers:
    layer.trainable = False

In [10]:
x_features = m.predict(x=x)
x_features.shape

(1, 1, 1, 2048)

In [20]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
x = m.output
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

In [24]:
baseline_model = Model(inputs=m.input, outputs=prediction)
baseline_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [30]:
baseline_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

In [10]:
x_valid_bin = x_test.copy()
y_valid_bin = y_test==1

In [28]:
import pandas as pd
hist = baseline_model.fit(x=x_train_bin, y=y_train_bin, validation_data=(x_valid_bin, y_valid_bin), epochs=10, 
                          verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 311s 6ms/sample - loss: 0.3563 - val_loss: 0.3329
Epoch 2/10
50000/50000 [==============================] - 301s 6ms/sample - loss: 0.2802 - val_loss: 0.3123
Epoch 3/10
50000/50000 [==============================] - 287s 6ms/sample - loss: 0.2591 - val_loss: 0.3043
Epoch 4/10
50000/50000 [==============================] - 285s 6ms/sample - loss: 0.2529 - val_loss: 0.3054
Epoch 5/10
50000/50000 [==============================] - 285s 6ms/sample - loss: 0.2444 - val_loss: 0.3040
Epoch 6/10
50000/50000 [==============================] - 259s 5ms/sample - loss: 0.2402 - val_loss: 0.3084
Epoch 7/10
50000/50000 [==============================] - 291s 6ms/sample - loss: 0.2356 - val_loss: 0.3070
Epoch 8/10
50000/50000 [==============================] - 283s 6ms/sample - loss: 0.2309 - val_loss: 0.3096
Epoch 9/10
50000/50000 [==============================] - 298s 6ms/sample - loss: 0.22

In [31]:
baseline_model.evaluate(x=x_valid_bin, y=y_valid_bin)

10000/10000 [==============================] - 40s 4ms/sample - loss: 0.3153 - acc: 0.8947


[0.3152909840464592, 0.8947]

### train the bin classifier with skewed data

In [6]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
feats = m.output

In [7]:
x = GlobalAveragePooling2D()(feats)
x = Dense(10, activation='relu')(x)
y = Dense(1, activation='sigmoid')(x)
test_model = Model(inputs=m.inputs, outputs=y)

In [8]:
test_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

In [38]:
mask = (y_train==1)
mask = np.squeeze(mask)
mask.shape

(50000,)

In [39]:
x_train_bin_sess1 = x_train_bin[mask]
x_train_bin_sess1.shape

(5000, 32, 32, 3)

In [40]:
y_train_bin_sess1 = y_train_bin[mask]
y_train_bin_sess1.shape

(5000, 1)

In [42]:
test_model.fit(x=x_train_bin_sess1, y=y_train_bin_sess1, epochs=10, verbose=1)

Epoch 1/10
5000/5000 [==============================] - 29s 6ms/sample - loss: 0.2659 - acc: 0.9024
Epoch 2/10
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0758 - acc: 0.9954
Epoch 3/10
5000/5000 [==============================] - 31s 6ms/sample - loss: 0.0459 - acc: 0.9988
Epoch 4/10
5000/5000 [==============================] - 26s 5ms/sample - loss: 0.0351 - acc: 0.9992
Epoch 5/10
5000/5000 [==============================] - 33s 7ms/sample - loss: 0.0271 - acc: 0.9998
Epoch 6/10
5000/5000 [==============================] - 33s 7ms/sample - loss: 0.0216 - acc: 1.0000
Epoch 7/10
5000/5000 [==============================] - 32s 6ms/sample - loss: 0.0183 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 32s 6ms/sample - loss: 0.0158 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0141 - acc: 1.0000
Epoch 10/10
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0122 - acc: 1.0000

In [43]:
test_model.evaluate(x=x_valid_bin, y=y_valid_bin)

10000/10000 [==============================] - 59s 6ms/sample - loss: 10.2037 - acc: 0.1000


[10.203698847961427, 0.1]

Appearantly, that wont work since this model is only taught by possitive examples. An auto negative example gens seems to be necessary. Just keep stream in negative samples and see what happens

In [56]:
# this time stream in data with 2nd class
mask = y_train==2
mask = np.squeeze(mask)
x_train_bin_sess2 = x_train_bin[mask]
y_train_bin_sess2 = y_train_bin[mask]
mask.shape, x_train_bin_sess2.shape, y_train_bin_sess2.shape

((50000,), (5000, 32, 32, 3), (5000, 1))

In [57]:
test_model.fit(x=x_train_bin_sess2, y=y_train_bin_sess2, epochs=10, verbose=1)

Epoch 1/10
5000/5000 [==============================] - 24s 5ms/sample - loss: 1.5232 - acc: 0.4612
Epoch 2/10
5000/5000 [==============================] - 24s 5ms/sample - loss: 0.1376 - acc: 0.9800
Epoch 3/10
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0605 - acc: 0.9972
Epoch 4/10
5000/5000 [==============================] - 34s 7ms/sample - loss: 0.0373 - acc: 0.9996
Epoch 5/10
5000/5000 [==============================] - 27s 5ms/sample - loss: 0.0281 - acc: 0.9996
Epoch 6/10
5000/5000 [==============================] - 26s 5ms/sample - loss: 0.0216 - acc: 0.9998
Epoch 7/10
5000/5000 [==============================] - 26s 5ms/sample - loss: 0.0175 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 30s 6ms/sample - loss: 0.0145 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 28s 6ms/sample - loss: 0.0130 - acc: 1.0000
Epoch 10/10
5000/5000 [==============================] - 27s 5ms/sample - loss: 0.0117 - acc: 1.0000

In [59]:
testout = test_model.predict(x_valid_bin)

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

The model simply goes to the other extreme end

In [63]:
np.sum(testout>0.1)

1

Even in bin classifier, skewed data hurt model badly

Next, we create 'perfect' learning sessions manually trying to defeat the baseline model (89% valid acc). The purpose of this experiment is to figure out whether extra negtive samples help with the model

In [83]:
mark = ((y_train==0) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess1 = x_train_bin[mark]
y_train_bin_sess1 = y_train_bin[mark]
x_train_bin_sess1.shape, y_train_bin_sess1.shape

((10000, 32, 32, 3), (10000, 1))

In [84]:
mark = ((y_train==2) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess2 = x_train_bin[mark]
y_train_bin_sess2 = y_train_bin[mark]
x_train_bin_sess2.shape, y_train_bin_sess2.shape

((10000, 32, 32, 3), (10000, 1))

In [85]:
mark = ((y_train==3) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess3 = x_train_bin[mark]
y_train_bin_sess3 = y_train_bin[mark]
x_train_bin_sess3.shape, y_train_bin_sess3.shape

((10000, 32, 32, 3), (10000, 1))

In [86]:
mark = ((y_train==4) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess4 = x_train_bin[mark]
y_train_bin_sess4 = y_train_bin[mark]
x_train_bin_sess4.shape, y_train_bin_sess4.shape

((10000, 32, 32, 3), (10000, 1))

In [87]:
mark = ((y_train==5) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess5 = x_train_bin[mark]
y_train_bin_sess5 = y_train_bin[mark]
x_train_bin_sess5.shape, y_train_bin_sess5.shape

((10000, 32, 32, 3), (10000, 1))

In [88]:
mark = ((y_train==6) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess6 = x_train_bin[mark]
y_train_bin_sess6 = y_train_bin[mark]
x_train_bin_sess6.shape, y_train_bin_sess6.shape

((10000, 32, 32, 3), (10000, 1))

In [89]:
mark = ((y_train==7) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess7 = x_train_bin[mark]
y_train_bin_sess7 = y_train_bin[mark]
x_train_bin_sess7.shape, y_train_bin_sess7.shape

((10000, 32, 32, 3), (10000, 1))

In [90]:
mark = ((y_train==8) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess8 = x_train_bin[mark]
y_train_bin_sess8 = y_train_bin[mark]
x_train_bin_sess8.shape, y_train_bin_sess8.shape

((10000, 32, 32, 3), (10000, 1))

In [91]:
mark = ((y_train==9) + (y_train==1))
mark = np.squeeze(mark)
x_train_bin_sess9 = x_train_bin[mark]
y_train_bin_sess9 = y_train_bin[mark]
x_train_bin_sess9.shape, y_train_bin_sess9.shape

((10000, 32, 32, 3), (10000, 1))

In [92]:
test_model.fit(x=x_train_bin_sess1, y=y_train_bin_sess1, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess2, y=y_train_bin_sess2, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess3, y=y_train_bin_sess3, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess4, y=y_train_bin_sess4, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess5, y=y_train_bin_sess5, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess6, y=y_train_bin_sess6, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess7, y=y_train_bin_sess7, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess8, y=y_train_bin_sess8, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)
test_model.fit(x=x_train_bin_sess9, y=y_train_bin_sess9, validation_data=(x_valid_bin, y_valid_bin), epochs=10, verbose=1)

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 94s 9ms/sample - loss: 0.9293 - acc: 0.6161 - val_loss: 0.7708 - val_acc: 0.4776
Epoch 2/10
10000/10000 [==============================] - 114s 11ms/sample - loss: 0.5384 - acc: 0.7344 - val_loss: 0.9812 - val_acc: 0.3619
Epoch 3/10
10000/10000 [==============================] - 84s 8ms/sample - loss: 0.4727 - acc: 0.7796 - val_loss: 1.1695 - val_acc: 0.3572
Epoch 4/10
10000/10000 [==============================] - 85s 9ms/sample - loss: 0.4298 - acc: 0.8125 - val_loss: 1.3211 - val_acc: 0.3592
Epoch 5/10
10000/10000 [==============================] - 108s 11ms/sample - loss: 0.4166 - acc: 0.8128 - val_loss: 1.4860 - val_acc: 0.3278
Epoch 6/10
10000/10000 [==============================] - 93s 9ms/sample - loss: 0.4001 - acc: 0.8269 - val_loss: 1.5323 - val_acc: 0.3317
Epoch 7/10
10000/10000 [==============================] - 84s 8ms/sample - loss: 0.3840 - acc: 0.8347 - val_loss

10000/10000 [==============================] - 85s 9ms/sample - loss: 0.2479 - acc: 0.9008 - val_loss: 3.7835 - val_acc: 0.2318
Epoch 8/10
10000/10000 [==============================] - 86s 9ms/sample - loss: 0.2423 - acc: 0.9080 - val_loss: 3.8170 - val_acc: 0.2281
Epoch 9/10
10000/10000 [==============================] - 88s 9ms/sample - loss: 0.2414 - acc: 0.9066 - val_loss: 3.9245 - val_acc: 0.2210
Epoch 10/10
10000/10000 [==============================] - 84s 8ms/sample - loss: 0.2476 - acc: 0.8985 - val_loss: 3.9964 - val_acc: 0.2121
Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 85s 8ms/sample - loss: 0.4582 - acc: 0.8030 - val_loss: 3.1164 - val_acc: 0.2342
Epoch 2/10
10000/10000 [==============================] - 85s 9ms/sample - loss: 0.3894 - acc: 0.8235 - val_loss: 2.6408 - val_acc: 0.2705
Epoch 3/10
10000/10000 [==============================] - 104s 10ms/sample - loss: 0.3572 - acc: 0.8427 - val_loss: 2.4556 - v